In [17]:
def create_netcdf(inyear, outyear, nmonths, pressin, lonin, latin, tracer):
    from netcdftime import utime, JulianDayFromDate,DateFromJulianDay, date2index
    #from netcdftime import datetime as datetimex,num2date
    from netCDF4 import Dataset
    import time
    from netCDF4 import num2date, date2num
    # create a series of dates for the netcdf file in times.
    from datetime import datetime, timedelta
    from netCDF4 import num2date, date2num
    
    field_o = np.load('ordonez_res'+str(inyear)+'.npy')
    field_l = np.load('liang_res'+str(inyear)+'.npy')
    field_z = np.load('ziska_res'+str(inyear)+'.npy')
    nmonths1, nlevs1, ny1, nx1 = np.shape(field_o)
    inyear=inyear+2000
    dataset = Dataset('N48_L60_ch2br2_'+str(outyear)+'_tracer'+str(tracer)+'.nc', 'w', format='NETCDF4')

    # Global Attributes
    dataset.description = 'TRANSCOM UMUKCA 7.3 N48L60'
    dataset.history = 'Created ' + time.ctime(time.time())
    dataset.source = ''
    #dataset.Conventions = 'CF-1.0'
    #dataset.standard_name_vocabulary='CF-1.0'

    time  = dataset.createDimension('time', None)
    time  = dataset.createVariable( 'time', 'f8', ('time',)) 
    time.standard_name = 'time'
    time.long_name = 'time'
    time.units = 'days since 1991-09-01 00:00:00'
    time.calendar = '360_day'
    time.axis = 'T'
    
    dates = []
    times = []
    for n in range (0,12):
        # nasty fix for the fact that datetime.timedelta only works on 365 day calendar
        new_date =  date2num(datetime(inyear,1,16),units=time.units, calendar=time.calendar)+n*30
        dates.append(num2date(new_date, units=time.units, calendar=time.calendar))
        
    time[:] = date2num(dates,units=time.units,calendar=time.calendar)
    
    # Create HDF5 *format*, classic *model*

    height = dataset.createDimension('pressure', nplevs)
    height = dataset.createVariable('pressure',  np.int32,   ('pressure',))
    height.units = 'mbar'
    height.axis='Z'
    height.positive='up'
    height.standard_name = 'pressure_levels'

    lat   = dataset.createDimension('lat',  ny1)
    lat   = dataset.createVariable('lat',   np.float32, ('lat',))
    lat.standard_name = 'latitude'
    lat.units = 'degrees_north'
    lat.cartesian_axis = "Y"
    lat.axis = 'Y'
    lat.actual_range = -90.0, 90.0

    lon   = dataset.createDimension('lon', nx1)
    lon   = dataset.createVariable('lon',  np.float32, ('lon',))
    lon.standard_name = 'longitude'
    lon.units = 'degrees_east'
    lon.cartesian_axis = "X"
    lon.axis = "X"
    lat.actual_range = 0.0, 360.0
    if tracer == 4:
        #liang
        tracer4 = dataset.createVariable('Tracer4_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
        tracer4.units = 'kg kg-1'
        tracer4.standard_name='CH2Br2 mass mixing ratio'
        tracer4[:,:,:,:] = field_l[:,:,:,:]
    elif tracer ==5:
        #ordonez
        tracer5 = dataset.createVariable('Tracer5_mmr', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
        tracer5.units = 'kg kg-1'
        tracer5.standard_name='CH2Br2 mass mixing ratio'
        tracer5[:,:,:,:] = field_o[:,:,:,:]    
    elif tracer ==6:
        tracer6 = dataset.createVariable('Tracer6_mmr', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
        tracer6.units = 'kg kg-1'
        tracer6.standard_name='CH2Br2 mass mixing ratio'
        tracer6[:,:,:,:] = field_z[:,:,:,:]
        
    lon[:]   = lonin[:]
    lat[:]   = latin[:]
    height[:] = pressin[:]

    print ('success')
    print 'time values for outyear '+str(outyear) +' (in units %s) for outyear ' % time.units+'\n',time[:]
    dataset.close()

    return outyear

In [18]:
import subprocess as sb
import numpy as np
import netCDF4
from netcdftime import utime, JulianDayFromDate,DateFromJulianDay, date2index
#from netcdftime import datetime as datetimex, date2num, num2date
import time
from netCDF4 import num2date, date2num

press = np.array([1000,950,900,850,700,500,400,300,250,200,150,100,70,50,30,20,10])*100
nplevs= len(press)

filename='/Users/ptg21/data/projects/TRANSCOM/N48_ancil_creation/xjcira.p.nc'
ncfile2     = netCDF4.Dataset(filename)

lon2        = ncfile2.variables['longitude'][:]
lat2        = ncfile2.variables['latitude'][:]
pressure    = ncfile2.variables['p'][:]
times2      = ncfile2.variables['t']
#inyears = years = [1,2,3,4,5,6,7,8]#[1998]
#outyears= years = [1,2,3,4,5,6,7,8]#[1998]
#[3,95], [4,96], [5,97], [6,98], [7,99], [8,00],  [1,9],  [2,10],[3,11],[4,12],
outyears = [[1,1], [2,2], [3,3], [4,4], [5,5], [6,6], [7,7], [8,8]]

for yearpair in outyears:
    filename_to_remove =  'N48_L60_ch2br2_'+str(yearpair[1])+'.nc'
    print ('removing '+ filename_to_remove)
    sb.Popen(['rm', '-Rf', filename_to_remove]).wait()
for tracer in [4,5,6]:
    for yearpair in outyears:
        print yearpair
        created_for = create_netcdf(inyear = yearpair[0], outyear=yearpair[1], nmonths = times2,\
                      pressin = press/100. , lonin = lon2, latin = lat2, tracer = tracer)
        print
        print 'netcdf created for', created_for
        print
        

removing N48_L60_ch2br2_1.nc
removing N48_L60_ch2br2_2.nc
removing N48_L60_ch2br2_3.nc
removing N48_L60_ch2br2_4.nc
removing N48_L60_ch2br2_5.nc
removing N48_L60_ch2br2_6.nc
removing N48_L60_ch2br2_7.nc
removing N48_L60_ch2br2_8.nc
[1, 1]
success
time values for outyear 1 (in units days since 1991-09-01 00:00:00) for outyear 
[ 3375.  3405.  3435.  3465.  3495.  3525.  3555.  3585.  3615.  3645.
  3675.  3705.]

netcdf created for 1

[2, 2]
success
time values for outyear 2 (in units days since 1991-09-01 00:00:00) for outyear 
[ 3735.  3765.  3795.  3825.  3855.  3885.  3915.  3945.  3975.  4005.
  4035.  4065.]

netcdf created for 2

[3, 3]
success
time values for outyear 3 (in units days since 1991-09-01 00:00:00) for outyear 
[ 4095.  4125.  4155.  4185.  4215.  4245.  4275.  4305.  4335.  4365.
  4395.  4425.]

netcdf created for 3

[4, 4]
success
time values for outyear 4 (in units days since 1991-09-01 00:00:00) for outyear 
[ 4455.  4485.  4515.  4545.  4575.  4605.  4635.  466

In [19]:
%%bash
ls -ltr *.nc

-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_8_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_7_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_6_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_5_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_4_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931358 21 Aug 12:38 N48_L60_chbr3_3_tracer5.nc
-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_3_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931358 21 Aug 12:38 N48_L60_chbr3_2_tracer5.nc
-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_2_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931358 21 Aug 12:38 N48_L60_chbr3_1_tracer5.nc
-rw-r--r--@ 1 ptg21  staff  9931359 21 Aug 12:38 N48_L60_chbr3_1_tracer4.nc
-rw-r--r--@ 1 ptg21  staff  9931358 21 Aug 12:38 N48_L60_chbr3_8_tracer6.nc
-rw-r--r--@ 1 ptg21  staff  9931358 21 Aug 12:38 N48_L60_chbr3_8_tracer5.nc
-rw-r--r--@ 

In [22]:
%%bash
rm -Rf N48_L60_ch2br2_tracer4.nc
ncrcat N48_L60_ch2br2_{1,2,3,4,5,6,7,8}_tracer4.nc N48_L60_ch2br2_tracer4.nc

rm -Rf N48_L60_ch2br2_tracer5.nc
ncrcat N48_L60_ch2br2_{1,2,3,4,5,6,7,8}_tracer5.nc N48_L60_ch2br2_tracer5.nc

rm -Rf N48_L60_ch2br2_tracer6.nc
ncrcat N48_L60_ch2br2_{1,2,3,4,5,6,7,8}_tracer6.nc N48_L60_ch2br2_tracer6.nc


In [23]:
%%bash
ncdump -h N48_L60_ch2br2_tracer6.nc

netcdf N48_L60_ch2br2_tracer6 {
dimensions:
	time = UNLIMITED ; // (96 currently)
	pressure = 17 ;
	lat = 73 ;
	lon = 96 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:units = "days since 1991-09-01 00:00:00" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
	int pressure(pressure) ;
		pressure:units = "mbar" ;
		pressure:axis = "Z" ;
		pressure:positive = "up" ;
		pressure:standard_name = "pressure_levels" ;
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:cartesian_axis = "Y" ;
		lat:axis = "Y" ;
		lat:actual_range = 0., 360. ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:cartesian_axis = "X" ;
		lon:axis = "X" ;
	float Tracer6_mmr(time, pressure, lat, lon) ;
		Tracer6_mmr:_FillValue = -999.f ;
		Tracer6_mmr:units = "kg kg-1" ;
		Tracer6_mmr:standard_name = "CH2Br2 mass mixing ratio" ;

// global attributes:
		:description = "TRANSCOM UMUKCA 

In [24]:
%%bash
cp N48_L60_ch2br2_tracer4.nc mm.UMUKCA.UCam.4.nc
cp N48_L60_ch2br2_tracer5.nc mm.UMUKCA.UCam.5.nc
cp N48_L60_ch2br2_tracer6.nc mm.UMUKCA.UCam.6.nc

In [25]:
%%bash
ls

N48_L60_ch2br2_1_tracer4.nc
N48_L60_ch2br2_1_tracer5.nc
N48_L60_ch2br2_1_tracer6.nc
N48_L60_ch2br2_2_tracer4.nc
N48_L60_ch2br2_2_tracer5.nc
N48_L60_ch2br2_2_tracer6.nc
N48_L60_ch2br2_3_tracer4.nc
N48_L60_ch2br2_3_tracer5.nc
N48_L60_ch2br2_3_tracer6.nc
N48_L60_ch2br2_4_tracer4.nc
N48_L60_ch2br2_4_tracer5.nc
N48_L60_ch2br2_4_tracer6.nc
N48_L60_ch2br2_5_tracer4.nc
N48_L60_ch2br2_5_tracer5.nc
N48_L60_ch2br2_5_tracer6.nc
N48_L60_ch2br2_6_tracer4.nc
N48_L60_ch2br2_6_tracer5.nc
N48_L60_ch2br2_6_tracer6.nc
N48_L60_ch2br2_7_tracer4.nc
N48_L60_ch2br2_7_tracer5.nc
N48_L60_ch2br2_7_tracer6.nc
N48_L60_ch2br2_8_tracer4.nc
N48_L60_ch2br2_8_tracer5.nc
N48_L60_ch2br2_8_tracer6.nc
N48_L60_ch2br2_tracer4.nc
N48_L60_ch2br2_tracer5.nc
N48_L60_ch2br2_tracer6.nc
N48_L60_chbr3_1_tracer4.nc
N48_L60_chbr3_1_tracer5.nc
N48_L60_chbr3_1_tracer6.nc
N48_L60_chbr3_2_tracer4.nc
N48_L60_chbr3_2_tracer5.nc
N48_L60_chbr3_2_tracer6.nc
N48_L60_chbr3_3_tracer4.nc
N48_L60_chbr3_3_tracer5.nc
N48_L60_chbr3_3_tracer6.nc
N48_L60